# Import Required Libraries

In [2]:
import os

import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

# Define Constants
Define constants such as the size of the training set.

In [3]:
# Define Constants

# The number of training examples to use
TRAINING_SIZE = 60000

# The fraction of the training data to use for validation
SPLIT = 0.8

# The number of test examples to use
TEST_SIZE = 10000

# The number of classes in the dataset (10 digits from 0 to 9)
NUM_CLASSES = 10

# The number of epochs to train for
EPOCHS = 10

# The batch size for training
BATCH_SIZE = 128

# The dimensions of the images in the dataset (28x28 pixels)
IMG_ROWS, IMG_COLS = 28, 28

# Input shape for the model
INPUT_SHAPE = IMG_ROWS * IMG_COLS

# Prepare and Preprocess MNIST Data
Load the MNIST dataset, reshape, normalize, shuffle, and split it into train and test sets.

In [4]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the data to fit the model
x_train = x_train.reshape(x_train.shape[0], INPUT_SHAPE)
x_test = x_test.reshape(x_test.shape[0], INPUT_SHAPE)

# Normalize the pixel values from a scale out of 255 to a scale out of 1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Shuffle the training dataset (for better randomization)
shuffle_index = np.random.permutation(TRAINING_SIZE)
x_train, y_train = x_train[shuffle_index], y_train[shuffle_index]

# Split the data into train and test sets
index_to_split = int(TRAINING_SIZE * SPLIT)
x_train, x_val = x_train[:index_to_split].T, x_train[index_to_split:].T
y_train, y_val = y_train[:index_to_split].T, y_train[index_to_split:].T
x_test = x_test.T

# One-hot encode the labels and transpose
y_train = np.eye(NUM_CLASSES)[y_train].T
y_val = np.eye(NUM_CLASSES)[y_val].T
y_test = np.eye(NUM_CLASSES)[y_test].T

# Print sizes
print('x_train:', x_train.shape)
print('x_val:', x_val.shape)
print('x_test:', x_test.shape)
print('y_train:', y_train.shape)
print('y_val:', y_val.shape)
print('y_test:', y_test.shape)

11490434/11490434 [==============================] - 1s 0us/step
x_train: (784, 48000)
x_val: (784, 12000)
x_test: (784, 10000)
y_train: (10, 48000)
y_val: (10, 12000)
y_test: (10, 10000)


# Train First Neural Network with Dense Layers

In [5]:
import os
import sys

sys.path.append(os.path.join(os.path.curdir, '..'))


In [6]:
# print Shape x_train

print(x_train.shape)

(784, 48000)


In [111]:
import net_modules.models as models
import net_modules.layers as layers
import net_modules.losses as losses

def reload_net():
  import importlib
  importlib.reload(models)
  importlib.reload(layers)
  importlib.reload(losses)

In [ ]:

model = models.Sequential()
model.add(layers.Dense(INPUT_SHAPE, 64))
model.add(layers.ELU())
model.add(layers.Dense(64, NUM_CLASSES))
model.add(layers.Softmax(cross_entropy_loss=True))

model.compile(loss=losses.CrossEntropy())

model.fit(x_train, y_train, epochs=100, learning_rate=0.1)

Epoch 1/100, Error: 14.575444876398048
Epoch 2/100, Error: 13.958183135382571
Epoch 3/100, Error: 13.433943218993994
Epoch 4/100, Error: 12.957158196915044
Epoch 5/100, Error: 12.510193142599832
Epoch 6/100, Error: 12.085568327935993
Epoch 7/100, Error: 11.679715177188921
Epoch 8/100, Error: 11.290691392574152
Epoch 9/100, Error: 10.917339026316618
Epoch 10/100, Error: 10.558952614295116
Epoch 11/100, Error: 10.21510356845411
Epoch 12/100, Error: 9.885500852500753
Epoch 13/100, Error: 9.569916135561844
Epoch 14/100, Error: 9.268124793183224
Epoch 15/100, Error: 8.979884282264786
Epoch 16/100, Error: 8.704921908533994
Epoch 17/100, Error: 8.442928344335462
Epoch 18/100, Error: 8.193556385990444
Epoch 19/100, Error: 7.956412440246552
Epoch 20/100, Error: 7.731068599566268
Epoch 21/100, Error: 7.517079100411167
Epoch 22/100, Error: 7.313977500378194
Epoch 23/100, Error: 7.12128787658128
Epoch 24/100, Error: 6.938528861284015
Epoch 25/100, Error: 6.765217969186999
Epoch 26/100, Error: 6.60

In [ ]:
model.evaluate(x_train, y_train, metrics=['accuracy'])

loss (crossentropy): 3.0444, accuracy: 0.8735


{'loss': 3.0444232659482986, 'accuracy': 0.8735208333333333}

In [ ]:
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=0)
y_true = np.argmax(y_test, axis=0)

for i in range(10):
    print(f'Predicted: {y_pred[i]}, True: {y_true[i]}')

Predicted: 7, True: 7
Predicted: 2, True: 2
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 4, True: 4
Predicted: 1, True: 1
Predicted: 4, True: 4
Predicted: 9, True: 9
Predicted: 6, True: 5
Predicted: 9, True: 9


# Train Second Neural Network with Convolutional Layers

In [36]:
import numpy as np



x_train_reshaped = x_train.reshape(28, 28, 1, -1)
x_val_reshaped = x_val.reshape(28, 28, 1, -1)
x_test_reshaped = x_test.reshape(28, 28, 1, -1)

print('x_train reshaped:', x_train_reshaped)
print('x_val reshaped:', x_val_reshaped.shape)
print('x_test reshaped:', x_test_reshaped.shape)

x_train reshaped: 1.0
x_val reshaped: (28, 28, 1, 12000)
x_test reshaped: (28, 28, 1, 10000)


In [68]:
def print_array_statistics(arr):
    print("Array Statistics:")
    print("Mean: ", np.mean(arr))
    print("Median: ", np.median(arr))
    print("Standard Deviation: ", np.std(arr))
    print("Minimum: ", np.min(arr))
    print("Maximum: ", np.max(arr))

In [113]:
reload_net()

In [116]:
model_cnn = models.Sequential()
model_cnn.add(layers.Conv2D(input_shape=(28, 28, 1),
                            kernel_size=(3,3),
                            number_filters=8,
                            initialization='lecun_uniform'))
model_cnn.add(layers.ReLU())
model_cnn.add(layers.Flatten())
model_cnn.add(layers.Dense(5408, 64))
model_cnn.add(layers.ReLU())
model_cnn.add(layers.Dense(64, NUM_CLASSES))
model_cnn.add(layers.ReLU())
model_cnn.add(layers.Softmax(cross_entropy_loss=True))
model_cnn.compile(loss=losses.CrossEntropy())
model_cnn.fit(x_train_reshaped, y_train, epochs=5, learning_rate=0.1, batch_size=512)

Epoch 1/5, Batch 1/93, Error: 4.21795139507046
Epoch 1/5, Batch 2/93, Error: 4.126688048764014
Epoch 1/5, Batch 3/93, Error: 4.047694272738991
Epoch 1/5, Batch 4/93, Error: 4.005149918712399
Epoch 1/5, Batch 5/93, Error: 3.991281284280523
Epoch 1/5, Batch 6/93, Error: 4.026851570192947
Epoch 1/5, Batch 7/93, Error: 3.8066544750758617
Epoch 1/5, Batch 8/93, Error: 3.5491648286153614
Epoch 1/5, Batch 9/93, Error: 3.4737659560953844
Epoch 1/5, Batch 10/93, Error: 3.4192135561350034
Epoch 1/5, Batch 11/93, Error: 3.3570595280379427
Epoch 1/5, Batch 12/93, Error: 3.3802742479853927
Epoch 1/5, Batch 13/93, Error: 3.243006780076452
Epoch 1/5, Batch 14/93, Error: 3.168080194592137
Epoch 1/5, Batch 15/93, Error: 2.9250313039713975
Epoch 1/5, Batch 16/93, Error: 2.528286280556152
Epoch 1/5, Batch 17/93, Error: 2.3860712972590883
Epoch 1/5, Batch 18/93, Error: 2.266542094482552
Epoch 1/5, Batch 19/93, Error: 2.102016180198955
Epoch 1/5, Batch 20/93, Error: 2.6625326803873044
Epoch 1/5, Batch 21/9

In [117]:
y_preds_cnn = model_cnn.predict(x_test_reshaped)

for elem in y_preds_cnn[:,10]:
  print(elem)

0.99992228783051
8.632163455222131e-07
4.1207451509426534e-05
8.632163455222131e-07
8.632163455222131e-07
2.8836632626932776e-05
2.488787280272025e-06
8.632163455222131e-07
8.632163455222131e-07
8.632163455222131e-07


In [118]:
model_cnn.evaluate(x_test_reshaped, y_test, metrics=['accuracy'])

loss (crossentropy): 5.3113, accuracy: 0.9572


{'loss': 5.311331553383126, 'accuracy': 0.9572}

In [120]:
y_preds_cnn = np.argmax(y_preds_cnn, axis=0)
y_true = np.argmax(y_test, axis=0)

for i in range(10):
    print(f'Predicted: {y_preds_cnn[i]}, True: {y_true[i]}')

Predicted: 7, True: 7
Predicted: 2, True: 2
Predicted: 1, True: 1
Predicted: 0, True: 0
Predicted: 4, True: 4
Predicted: 1, True: 1
Predicted: 4, True: 4
Predicted: 9, True: 9
Predicted: 6, True: 5
Predicted: 9, True: 9
